# 1 Data Wrangling

## Introduction

In the National Football League (NFL) one team has 4 downs to go 10 yards which would reset their down count and allow them to try another 4 times to get another 4 downs. Getting 10 yards within the set of downs is vital to the game as it allows that team to progress down the field and potentially score. But what if you could find a way to predict the play on 3rd downs (as those are typically the most important) based off of previous data. 

The data can be found here: https://www.dolthub.com/repositories/Liquidata/nfl-play-by-play/data/master/plays 

In [1]:
# import pandas as pd 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
#import raw data
YEARS = [2019,2018,2017]

data = pd.DataFrame()

for i in YEARS:  
    #low_memory=False eliminates a warning
    i_data = pd.read_csv('https://github.com/nflverse/nflfastR-data/blob/master/data/' \
                         'play_by_play_' + str(i) + '.csv.gz?raw=True',
                         compression='gzip', low_memory=False)

    #sort=True eliminates a warning and alphabetically sorts columns
    data = data.append(i_data, sort=True)

#Give each row a unique index
data.reset_index(drop=True, inplace=True)

## Explore Raw Data

In [5]:
pd.set_option('display.max_columns', None)
with pd.option_context('display.max_rows',5):
    display(data)

,aborted_play,air_epa,air_wpa,air_yards,assist_tackle,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tackle_2_player_id,assist_tackle_2_player_name,assist_tackle_2_team,assist_tackle_3_player_id,assist_tackle_3_player_name,assist_tackle_3_team,assist_tackle_4_player_id,assist_tackle_4_player_name,assist_tackle_4_team,away_coach,away_score,away_team,away_timeouts_remaining,away_wp,away_wp_post,blocked_player_id,blocked_player_name,comp_air_epa,comp_air_wpa,comp_yac_epa,comp_yac_wpa,complete_pass,cp,cpoe,def_wp,defensive_extra_point_attempt,defensive_extra_point_conv,defensive_two_point_attempt,defensive_two_point_conv,defteam,defteam_score,defteam_score_post,defteam_timeouts_remaining,desc,div_game,down,drive,drive_end_transition,drive_end_yard_line,drive_ended_with_score,drive_first_downs,drive_game_clock_end,drive_game_clock_start,drive_inside20,drive_play_count,drive_play_id_ended,drive_play_id_started,drive_quarter_end,drive_quarter_start,drive_real_start_time,drive_start_transition,drive_start_yard_line,drive_time_of_possession,drive_yards_penalized,end_clock_time,end_yard_line,ep,epa,extra_point_attempt,extra_point_prob,extra_point_result,fantasy,fantasy_id,fantasy_player_id,fantasy_player_name,fg_prob,field_goal_attempt,field_goal_result,first_down,first_down_pass,first_down_penalty,first_down_rush,fixed_drive,fixed_drive_result,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_1_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,forced_fumble_player_2_team,fourth_down_converted,fourth_down_failed,fumble,fumble_forced,fumble_lost,fumble_not_forced,fumble_out_of_bounds,fumble_recovery_1_player_id,fumble_recovery_1_player_name,fumble_recovery_1_team,fumble_recovery_1_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumbled_1_player_id,fumbled_1_player_name,fumbled_1_team,fumbled_2_player_id,fumbled_2_player_name,fumbled_2_team,game_date,game_half,game_id,game_seconds_remaining,game_stadium,goal_to_go,half_sack_1_player_id,half_sack_1_player_name,half_sack_2_player_id,half_sack_2_player_name,half_seconds_remaining,home_coach,home_opening_kickoff,home_score,home_team,home_timeouts_remaining,home_wp,home_wp_post,id,incomplete_pass,interception,interception_player_id,interception_player_name,jersey_number,kick_distance,kicker_player_id,kicker_player_name,kickoff_attempt,kickoff_downed,kickoff_fair_catch,kickoff_in_endzone,kickoff_inside_twenty,kickoff_out_of_bounds,kickoff_returner_player_id,kickoff_returner_player_name,lateral_interception_player_id,lateral_interception_player_name,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,lateral_receiver_player_id,lateral_receiver_player_name,lateral_receiving_yards,lateral_reception,lateral_recovery,lateral_return,lateral_rush,lateral_rusher_player_id,lateral_rusher_player_name,lateral_rushing_yards,lateral_sack_player_id,lateral_sack_player_name,location,name,nfl_api_id,no_huddle,no_score_prob,old_game_id,opp_fg_prob,opp_safety_prob,opp_td_prob,order_sequence,out_of_bounds,own_kickoff_recovery,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,own_kickoff_recovery_td,pass,pass_attempt,pass_defense_1_player_id,pass_defense_1_player_name,pass_defense_2_player_id,pass_defense_2_player_name,pass_length,pass_location,pass_oe,pass_touchdown,passer,passer_id,passer_jersey_number,passer_player_id,passer_player_name,passing_yards,penalty,penalty_player_id,penalty_player_name,penalty_team,penalty_type,penalty_yards,play,play_clock,play_deleted,play_id,play_type,play_type_nfl,posteam,posteam_score,posteam_score_post,posteam_timeouts_remaining,posteam_type,punt_attempt,punt_blocked,punt_downed,punt_fair_catch,punt_in_endzone,punt_inside_twenty,punt_out_of_bounds,punt_returner_player_id,punt_returner_player_name,punter_player_id,punter_pla

In [6]:
#Select important data columns 
important_columns = data[['air_epa', 'air_wpa', 'air_yards','away_score', 'away_team', 'posteam', 'complete_pass','defteam', 'defteam_score',
                          'defteam_score_post','down','drive_start_yard_line', 'drive_time_of_possession', 'first_down_pass',
                          'first_down_penalty', 'first_down_rush','game_half','game_seconds_remaining','home_score',
                          'incomplete_pass', 'interception','lateral_receiving_yards','lateral_reception','lateral_rush',
                          'lateral_rushing_yards', 'no_huddle','no_score_prob','pass_attempt','pass_length', 'pass_location',
                          'passing_yards','play_type', 'play_type_nfl', 'posteam_score', 'qb_kneel', 'qb_scramble',
                          'qb_spike','qtr', 'quarter_end', 'quarter_seconds_remaining','receiving_yards','run_gap',
                          'run_location', 'rush', 'rush_attempt', 'rushing_yards','score_differential', 'series',
                          'series_result', 'series_success', 'shotgun', 'side_of_field','success','third_down_converted',
                          'third_down_failed', 'time','timeout', 'timeout_team', 'touchdown', 'weather', 'wp', 'wpa', 
                          'xpass', 'xyac_epa', 'xyac_fd', 'xyac_mean_yardage', 'xyac_median_yardage', 'xyac_success',
                          'yardline_100', 'yards_after_catch', 'yards_gained', 'ydsnet', 'ydstogo', 'yrdln']]

In [9]:
third_downs = important_columns[important_columns.down == 3.0]
third_downs_with_nan.head()

,air_epa,air_wpa,air_yards,away_score,away_team,posteam,complete_pass,defteam,defteam_score,defteam_score_post,down,drive_start_yard_line,drive_time_of_possession,first_down_pass,first_down_penalty,first_down_rush,game_half,game_seconds_remaining,home_score,incomplete_pass,interception,lateral_receiving_yards,lateral_reception,lateral_rush,lateral_rushing_yards,no_huddle,no_score_prob,pass_attempt,pass_length,pass_location,passing_yards,play_type,play_type_nfl,posteam_score,qb_kneel,qb_scramble,qb_spike,qtr,quarter_end,quarter_seconds_remaining,receiving_yards,run_gap,run_location,rush,rush_attempt,rushing_yards,score_differential,series,series_result,series_success,shotgun,side_of_field,success,third_down_converted,third_down_failed,time,timeout,timeout_team,touchdown,weather,wp,wpa,xpass,xyac_epa,xyac_fd,xyac_mean_yardage,xyac_median_yardage,xyac_success,yardline_100,yards_after_catch,yards_gained,ydsnet,ydstogo,yrdln
4,NaN,NaN,NaN,12,ATL,ATL,0.0,MIN,0.0,0.0,3.0,ATL 25,2:07,0.0,0.0,0.0,Half1,3521.0,28,0.0,0.0,NaN,0.0,0.0,NaN,0,0.003781,0.0,NaN,NaN,NaN,run,RUSH,0.0,0,1,0,1,0,821.0,NaN,end,left,0,1.0,12.0,0.0,1,Punt,0,1,ATL,1.0,0.0,1.0,13:41,0.0,NaN,0.0,"N/A (Indoors) Temp: ° F, Wind: mph",0.379757,0.018007,0.933516,NaN,NaN,NaN,NaN,NaN,79.0,NaN,12.0,8.0,14,ATL 21
13,NaN,NaN,NaN,12,ATL,ATL,0.0,MIN,7.0,7.0,3.0,ATL 16,1:55,0.0,1.0,0.0,Half1,3323.0,28,0.0,0.0,NaN,0.0,0.0,NaN,0,0.005124,0.0,NaN,NaN,NaN,no_play,PENALTY,0.0,0,0,0,1,0,623.0,NaN,NaN,NaN,0,0.0,NaN,-7.0,3,First down,1,1,ATL,1.0,0.0,0.0,10:23,0.0,NaN,0.0,"N/A (Indoors) Temp: ° F, Wind: mph",0.231108,0.015072,0.918794,NaN,NaN,NaN,NaN,NaN,77.0,NaN,0.0,12.0,3,ATL 23
23,NaN,NaN,NaN,12,ATL,ATL,0.0,MIN,14.0,14.0,3.0,ATL 25,5:51,0.0,1.0,0.0,Half1,3123.0,28,0.0,0.0,NaN,0.0,0.0,NaN,0,0.010779,0.0,NaN,NaN,NaN,no_play,PENALTY,0.0,0,0,0,1,0,423.0,NaN,NaN,NaN,0,0.0,NaN,-14.0,7,First down,1,1,ATL,1.0,0.0,0.0,07:03,0.0,NaN,0.0,"N/A (Indoors) Temp: ° F, Wind: mph",0.121559,0.013919,0.969930,NaN,NaN,NaN,NaN,NaN,70.0,NaN,0.0,34.0,5,ATL 30
26,1.520888,0.0,5.0,12,ATL,ATL,1.0,MIN,14.0,14.0,3.0,ATL 25,5:51,1.0,0.0,0.0,Half1,3061.0,28,0.0,0.0,NaN,0.0,0.0,NaN,0,0.013355,1.0,short,left,6.0,pass,PASS,0.0,0,0,0,1,0,361.0,6.0,NaN,NaN,0,0.0,NaN,-14.0,8,First down,1,1,ATL,1.0,1.0,0.0,06:01,0.0,NaN,0.0,"N/A (Indoors) Temp: ° F, Wind: mph",0.128082,0.016882,0.967478,0.275905,0.974920,4.289433,2.0,0.97492,60.0,1.0,6.0,34.0,5,ATL 40
32,3.544151,0.0,21.0,12,ATL,ATL,0.0,MIN,14.0,14.0,3.0,ATL 25,5:51,0.0,0.0,0.0,Half1,2865.0,28,1.0,0.0,NaN,0.0,0.0,NaN,0,0.018408,1.0,deep,left,NaN,pass,PASS,0.0,0,0,0,1,0,165.0,NaN,NaN,NaN,0,0.0,NaN,-14.0,10,Punt,0,1,MIN,0.0,0.0,1.0,02:45,0.0,NaN,0.0,"N/A (Indoors) Temp: ° F, Wind: mph",0.117711,-0.012735,0.972965,0.323985,0.997187,3.835246,2.0,1.00000,41.0,NaN,0.0,34.0,19,MIN 41


In [10]:
#Explore data 
third_downs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22791 entries, 4 to 143896
Data columns (total 74 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   air_epa                    14704 non-null  float64
 1   air_wpa                    14704 non-null  float64
 2   air_yards                  14725 non-null  float64
 3   away_score                 22791 non-null  int64  
 4   away_team                  22791 non-null  object 
 5   posteam                    22791 non-null  object 
 6   complete_pass              22745 non-null  float64
 7   defteam                    22791 non-null  object 
 8   defteam_score              22791 non-null  float64
 9   defteam_score_post         22791 non-null  float64
 10  down                       22791 non-null  float64
 11  drive_start_yard_line      22791 non-null  object 
 12  drive_time_of_possession   22791 non-null  object 
 13  first_down_pass            22745 non-null  fl

In [11]:
#Remove columns with more than 50% NaN values
third_downs = third_downs_with_nan.columns[third_downs_with_nan.isnull().mean() < 0.5]

In [12]:
third_downs_df = data[third_downs]

In [13]:
third_downs_df = third_downs_df[third_downs_df.down == 3.0]

In [14]:
third_downs_df.head()

,air_epa,air_wpa,air_yards,away_score,away_team,posteam,complete_pass,defteam,defteam_score,defteam_score_post,down,drive_start_yard_line,drive_time_of_possession,first_down_pass,first_down_penalty,first_down_rush,game_half,game_seconds_remaining,home_score,incomplete_pass,interception,lateral_reception,lateral_rush,no_huddle,no_score_prob,pass_attempt,pass_length,pass_location,play_type,play_type_nfl,posteam_score,qb_kneel,qb_scramble,qb_spike,qtr,quarter_end,quarter_seconds_remaining,rush,rush_attempt,score_differential,series,series_result,series_success,shotgun,side_of_field,success,third_down_converted,third_down_failed,time,timeout,touchdown,weather,wp,wpa,xpass,xyac_epa,xyac_fd,xyac_mean_yardage,xyac_median_yardage,xyac_success,yardline_100,yards_gained,ydsnet,ydstogo,yrdln
4,NaN,NaN,NaN,12,ATL,ATL,0.0,MIN,0.0,0.0,3.0,ATL 25,2:07,0.0,0.0,0.0,Half1,3521.0,28,0.0,0.0,0.0,0.0,0,0.003781,0.0,NaN,NaN,run,RUSH,0.0,0,1,0,1,0,821.0,0,1.0,0.0,1,Punt,0,1,ATL,1.0,0.0,1.0,13:41,0.0,0.0,"N/A (Indoors) Temp: ° F, Wind: mph",0.379757,0.018007,0.933516,NaN,NaN,NaN,NaN,NaN,79.0,12.0,8.0,14,ATL 21
13,NaN,NaN,NaN,12,ATL,ATL,0.0,MIN,7.0,7.0,3.0,ATL 16,1:55,0.0,1.0,0.0,Half1,3323.0,28,0.0,0.0,0.0,0.0,0,0.005124,0.0,NaN,NaN,no_play,PENALTY,0.0,0,0,0,1,0,623.0,0,0.0,-7.0,3,First down,1,1,ATL,1.0,0.0,0.0,10:23,0.0,0.0,"N/A (Indoors) Temp: ° F, Wind: mph",0.231108,0.015072,0.918794,NaN,NaN,NaN,NaN,NaN,77.0,0.0,12.0,3,ATL 23
23,NaN,NaN,NaN,12,ATL,ATL,0.0,MIN,14.0,14.0,3.0,ATL 25,5:51,0.0,1.0,0.0,Half1,3123.0,28,0.0,0.0,0.0,0.0,0,0.010779,0.0,NaN,NaN,no_play,PENALTY,0.0,0,0,0,1,0,423.0,0,0.0,-14.0,7,First down,1,1,ATL,1.0,0.0,0.0,07:03,0.0,0.0,"N/A (Indoors) Temp: ° F, Wind: mph",0.121559,0.013919,0.969930,NaN,NaN,NaN,NaN,NaN,70.0,0.0,34.0,5,ATL 30
26,1.520888,0.0,5.0,12,ATL,ATL,1.0,MIN,14.0,14.0,3.0,ATL 25,5:51,1.0,0.0,0.0,Half1,3061.0,28,0.0,0.0,0.0,0.0,0,0.013355,1.0,short,left,pass,PASS,0.0,0,0,0,1,0,361.0,0,0.0,-14.0,8,First down,1,1,ATL,1.0,1.0,0.0,06:01,0.0,0.0,"N/A (Indoors) Temp: ° F, Wind: mph",0.128082,0.016882,0.967478,0.275905,0.974920,4.289433,2.0,0.97492,60.0,6.0,34.0,5,ATL 40
32,3.544151,0.0,21.0,12,ATL,ATL,0.0,MIN,14.0,14.0,3.0,ATL 25,5:51,0.0,0.0,0.0,Half1,2865.0,28,1.0,0.0,0.0,0.0,0,0.018408,1.0,deep,left,pass,PASS,0.0,0,0,0,1,0,165.0,0,0.0,-14.0,10,Punt,0,1,MIN,0.0,0.0,1.0,02:45,0.0,0.0,"N/A (Indoors) Temp: ° F, Wind: mph",0.117711,-0.012735,0.972965,0.323985,0.997187,3.835246,2.0,1.00000,41.0,0.0,34.0,19,MIN 41


In [15]:
third_downs_df = third_downs_df.reset_index()

In [16]:
third_downs_df.drop(columns=third_downs_df.columns[0], axis=1, inplace=True)

In [18]:
third_downs_df.describe().T

,count,mean,std,min,25%,50%,75%,max
air_epa,14704.0,0.780055,1.695466,-9.400001,-0.841593,1.065626,2.087206,6.761917
air_wpa,14704.0,0.004731,0.031003,-0.772345,0.000000,0.000000,0.000000,0.495123
air_yards,14725.0,8.851409,9.736584,-15.000000,3.000000,6.000000,13.000000,61.000000
away_score,22791.0,21.666360,9.923196,0.000000,15.000000,21.000000,28.000000,59.000000
complete_pass,22745.0,0.381403,0.485742,0.000000,0.000000,0.000000,1.000000,1.000000
defteam_score,22791.0,11.343952,9.975712,0.000000,3.000000,10.000000,17.000000,59.000000
defteam_score_post,22791.0,11.360976,9.982198,0.000000,3.000000,10.000000,17.000000,59.000000
down,22791.0,3.000000,0.000000,3.000000,3.000000,3.000000,3.000000,3.000000
first_down_pass,22745.0,0.257024,0.437002,0.000000,0.000000,0.000000,1.000000,1.000000
first_down_penalty,22745.0,0.044801,0.206871,0.000000,0.000000,0.000000,0.000000,1.000000


In [19]:
df = third_downs_df

In [20]:
df.head()

,air_epa,air_wpa,air_yards,away_score,away_team,posteam,complete_pass,defteam,defteam_score,defteam_score_post,down,drive_start_yard_line,drive_time_of_possession,first_down_pass,first_down_penalty,first_down_rush,game_half,game_seconds_remaining,home_score,incomplete_pass,interception,lateral_reception,lateral_rush,no_huddle,no_score_prob,pass_attempt,pass_length,pass_location,play_type,play_type_nfl,posteam_score,qb_kneel,qb_scramble,qb_spike,qtr,quarter_end,quarter_seconds_remaining,rush,rush_attempt,score_differential,series,series_result,series_success,shotgun,side_of_field,success,third_down_converted,third_down_failed,time,timeout,touchdown,weather,wp,wpa,xpass,xyac_epa,xyac_fd,xyac_mean_yardage,xyac_median_yardage,xyac_success,yardline_100,yards_gained,ydsnet,ydstogo,yrdln
0,NaN,NaN,NaN,12,ATL,ATL,0.0,MIN,0.0,0.0,3.0,ATL 25,2:07,0.0,0.0,0.0,Half1,3521.0,28,0.0,0.0,0.0,0.0,0,0.003781,0.0,NaN,NaN,run,RUSH,0.0,0,1,0,1,0,821.0,0,1.0,0.0,1,Punt,0,1,ATL,1.0,0.0,1.0,13:41,0.0,0.0,"N/A (Indoors) Temp: ° F, Wind: mph",0.379757,0.018007,0.933516,NaN,NaN,NaN,NaN,NaN,79.0,12.0,8.0,14,ATL 21
1,NaN,NaN,NaN,12,ATL,ATL,0.0,MIN,7.0,7.0,3.0,ATL 16,1:55,0.0,1.0,0.0,Half1,3323.0,28,0.0,0.0,0.0,0.0,0,0.005124,0.0,NaN,NaN,no_play,PENALTY,0.0,0,0,0,1,0,623.0,0,0.0,-7.0,3,First down,1,1,ATL,1.0,0.0,0.0,10:23,0.0,0.0,"N/A (Indoors) Temp: ° F, Wind: mph",0.231108,0.015072,0.918794,NaN,NaN,NaN,NaN,NaN,77.0,0.0,12.0,3,ATL 23
2,NaN,NaN,NaN,12,ATL,ATL,0.0,MIN,14.0,14.0,3.0,ATL 25,5:51,0.0,1.0,0.0,Half1,3123.0,28,0.0,0.0,0.0,0.0,0,0.010779,0.0,NaN,NaN,no_play,PENALTY,0.0,0,0,0,1,0,423.0,0,0.0,-14.0,7,First down,1,1,ATL,1.0,0.0,0.0,07:03,0.0,0.0,"N/A (Indoors) Temp: ° F, Wind: mph",0.121559,0.013919,0.969930,NaN,NaN,NaN,NaN,NaN,70.0,0.0,34.0,5,ATL 30
3,1.520888,0.0,5.0,12,ATL,ATL,1.0,MIN,14.0,14.0,3.0,ATL 25,5:51,1.0,0.0,0.0,Half1,3061.0,28,0.0,0.0,0.0,0.0,0,0.013355,1.0,short,left,pass,PASS,0.0,0,0,0,1,0,361.0,0,0.0,-14.0,8,First down,1,1,ATL,1.0,1.0,0.0,06:01,0.0,0.0,"N/A (Indoors) Temp: ° F, Wind: mph",0.128082,0.016882,0.967478,0.275905,0.974920,4.289433,2.0,0.97492,60.0,6.0,34.0,5,ATL 40
4,3.544151,0.0,21.0,12,ATL,ATL,0.0,MIN,14.0,14.0,3.0,ATL 25,5:51,0.0,0.0,0.0,Half1,2865.0,28,1.0,0.0,0.0,0.0,0,0.018408,1.0,deep,left,pass,PASS,0.0,0,0,0,1,0,165.0,0,0.0,-14.0,10,Punt,0,1,MIN,0.0,0.0,1.0,02:45,0.0,0.0,"N/A (Indoors) Temp: ° F, Wind: mph",0.117711,-0.012735,0.972965,0.323985,0.997187,3.835246,2.0,1.00000,41.0,0.0,34.0,19,MIN 41


In [21]:
third_downs_df.shape

(22791, 65)

In [22]:
# save the data to a new csv file
df.to_csv(r'C:\Users\geral\Documents\Springboard\Capstone_project_1\third_downs_df_1.cvs', index=False)